In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=fe7deda53dab17b2c4df93c973b6fac728e89ea2f52a65f7910afa69f488d1d9
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark_practice_3").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/22 08:19:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
# Get data into rdd
rdd = spark.sparkContext.textFile('/kaggle/input/logs-data-pyspark/ghtorrent-logs.txt')

In [5]:
# Q1: Count number of records
rdd.count()

9669788

In [6]:
rdd.getNumPartitions()

37

In [7]:
spark.sparkContext.defaultParallelism

4

In [8]:
rdd.getStorageLevel()

StorageLevel(False, False, False, False, 1)

In [9]:
from pyspark import StorageLevel
rdd.unpersist()
rdd.persist(StorageLevel.MEMORY_AND_DISK)
print(rdd.getStorageLevel())

Disk Memory Serialized 1x Replicated


In [10]:
spark.sparkContext.setLogLevel("ERROR")

In [11]:
# Q1: Count the number of records and get twenty records randomly
%time
# rdd.takeSample(False, 20) # take too much time
rdd.take(20)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 12.2 µs


['INFO, 2017-03-22T20:11:49+00:00, ghtorrent-31 -- ghtorrent.rb: Added pullreq_commit 244eeac28bf419642d5d5c3b388bd2999c8c72e6 to tgstation/tgstation -> 25341',
 'DEBUG, 2017-03-23T11:15:14+00:00, ghtorrent-30 -- retriever.rb: Commit mzvast/FlappyFrog -> 80bf5c5fde7be6274a2721422f4d9a773583f73c exists',
 'DEBUG, 2017-03-22T20:15:48+00:00, ghtorrent-35 -- ghtorrent.rb: Parent af8451e16e077f7c6cae3f98bf43bffaca562f88 for commit 2ef393531a3cfbecc69f17d2cedcc95662fae1e6 exists',
 'DEBUG, 2017-03-24T12:29:50+00:00, ghtorrent-49 -- ghtorrent.rb: Parent cf060bf3b789ac6391b2f7c1cdc34191c2bc773d for commit 8c924c1115e1abddcaddc27c6e7fd5806583ea90 exists',
 'DEBUG, 2017-03-23T09:00:44+00:00, ghtorrent-8 -- retriever.rb: Commit iamtheanon/d3 -> a7caf9375fe14d7235562af541fe9decf499bbfb exists',
 'DEBUG, 2017-03-24T10:52:47+00:00, ghtorrent-50 -- ghtorrent.rb: Repo alyuev/urantia-study-edition exists',
 'DEBUG, 2017-03-23T11:02:10+00:00, ghtorrent-45 -- retriever.rb: Commit heycalmdown/node-conflue

## **GHTorrent data format**
Every line of this log file includes:
1. Logging level, one of **DEBUG**, **INFO**, **WARN**, **ERROR**
2. A timestamp
3. The download id
4. The logging stage including at least one of the following names:
   - **event_processing**
   - **ght_data_retrieval**
   - **api_client**
   - **retriever**
   - **ghtorrent**


In [12]:
# Q2: Get the number of lines with both 'Transaction' and 'Repo' information
%time
import re

def collect_word_line(line):
    return re.compile(r"\w+").findall(line.lower())

collect_word_line('DEBUG, 2017-03-23T09:13:17+00:00, ghtorrent-9 -- retriever.rb: Repo xuminwlt -> tcc-transaction exists')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


['debug',
 '2017',
 '03',
 '23t09',
 '13',
 '17',
 '00',
 '00',
 'ghtorrent',
 '9',
 'retriever',
 'rb',
 'repo',
 'xuminwlt',
 'tcc',
 'transaction',
 'exists']

In [13]:
%time
find_repo = rdd.filter(lambda x: "repo" in collect_word_line(x))
find_repo.count()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


1846067

In [14]:
%time
find_trx = rdd.filter(lambda x: "transaction" in collect_word_line(x))
find_trx.count()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.78 µs


315750

In [15]:
find_trx.take(2)

['DEBUG, 2017-03-23T11:03:03+00:00, ghtorrent-39 -- ghtorrent.rb: Transaction committed (618 ms)',
 'DEBUG, 2017-03-23T09:23:35+00:00, ghtorrent-41 -- ghtorrent.rb: Transaction committed (505 ms)']

In [16]:
%time
rdd_repo_trx = find_repo.intersection(find_trx)
rdd_repo_trx.count()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs


19

In [17]:
rdd_repo_trx.collect()

['DEBUG, 2017-03-23T09:13:16+00:00, ghtorrent-9 -- ghtorrent.rb: Repo changmingxie/tcc-transaction exists',
 'DEBUG, 2017-03-23T09:13:26+00:00, ghtorrent-9 -- ghtorrent.rb: Association of commit 5793a7df39a26b46082cafb59f287e2dc2cf9796 with repo xuminwlt/tcc-transaction exists',
 'DEBUG, 2017-03-23T09:13:16+00:00, ghtorrent-9 -- ghtorrent.rb: Repo xuminwlt/tcc-transaction exists',
 'DEBUG, 2017-03-23T09:27:31+00:00, ghtorrent-24 -- ghtorrent.rb: Repo changmingxie/tcc-transaction exists',
 'INFO, 2017-03-23T09:13:16+00:00, ghtorrent-9 -- retriever.rb: Added repo xuminwlt -> tcc-transaction',
 'DEBUG, 2017-03-23T09:13:17+00:00, ghtorrent-9 -- ghtorrent.rb: Repo changmingxie/tcc-transaction exists',
 'DEBUG, 2017-03-23T11:09:37+00:00, ghtorrent-1 -- ghtorrent.rb: Repo pilkyoon/spring-transaction exists',
 'DEBUG, 2017-03-23T09:13:28+00:00, ghtorrent-9 -- ghtorrent.rb: Repo xuminwlt/tcc-transaction exists',
 'INFO, 2017-03-23T09:13:16+00:00, ghtorrent-9 -- ghtorrent.rb: Repo xuminwlt/tcc-t

In [18]:
%time
rdd.filter(lambda x: "url" in re.compile("\w+").findall(x.lower())).count()

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 11.7 µs


893474

In [19]:
str = 'WARN, 2017-03-23T13:00:25+00:00, ghtorrent-21 -- api_client.rb: Failed request. URL: https://api.github.com/repos/coreyfarrell/docs?per_page=100, Status code: 401, Status: Unauthorized, Access: 46f11b5791b, IP: 0.0.0.0, Remaining: 38'
regexp_url = r"http[s]?://(?:[-\w.]|(?:%[\da-zA-Z]{2,}))+"
# regexp_url = ""
re.findall(regexp_url, str)

['https://api.github.com']

In [20]:

def collected_word(line):
    return re.compile("\w+").findall(line.split(",")[2].lower())

collected_word(str)

['ghtorrent',
 '21',
 'api_client',
 'rb',
 'failed',
 'request',
 'url',
 'https',
 'api',
 'github',
 'com',
 'repos',
 'coreyfarrell',
 'docs',
 'per_page',
 '100']

In [21]:
rdd.filter(lambda x: (len(re.findall(regexp_url, x)) > 1) and ("warn" in x.split(",")[0])).count()

0

In [22]:
# Q3: Get the number of lines including web link for WARN logging
%time
rdd.filter(lambda x: (x.split(",")[0].lower())).filter(lambda x: len(re.findall(regexp_url, x)) > 1).count()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10.7 µs


57

In [23]:
%time
rdd_1 = rdd.filter(lambda x: ("warn" in x.split(",")[0].lower())).filter(lambda x: len(re.findall(regexp_url, x)) > 0)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.44 µs


In [24]:
%time
rdd_2 = rdd.filter(lambda x: ("url:" in x.lower()) and ("warn" in x.split(",")[0].lower()))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


In [25]:
%time
rdd_outer = rdd_1.subtract(rdd_2).union(rdd_2.subtract(rdd_1))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 10.7 µs


In [26]:
rdd_outer.count()

58

In [27]:
%time
rdd.filter(lambda x: (len(re.findall(regexp_url, x)) > 1) and ("warn" in x.split(",")[0].lower())).count()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.68 µs


57

In [28]:
# Q4: What is the most active 'downloader id' for 'failed' connections
def get_downloader(line):
    return line.split(",")[2].split("--")[0].strip()
    
get_downloader(str)                                                                        

'ghtorrent-21'

In [29]:
rdd_failed = rdd.filter(lambda x: ("failed" in x.lower())).map(lambda x: (get_downloader(x), 1)).reduceByKey(lambda x,y: x+y)
rdd_failed.sortBy(lambda x: x[1], ascending=False).first()  

('ghtorrent-13', 79654)

In [30]:
# Q5: What is the most active repository
def get_repo_name(line):
    return line.lower().split(": repo ")[1].split(" ")[0] 
str ='DEBUG, 2017-03-23T10:28:03+00:00, ghtorrent-33 -- ghtorrent.rb: Repo jwpttcg66/redis-game-transaction exists'
get_repo_name(str)


'jwpttcg66/redis-game-transaction'

In [31]:
rdd_most_active_repo = (rdd.filter(lambda x: ": repo " in x.lower())
                            .map(lambda x: (get_repo_name(x), 1))
                            .reduceByKey(lambda x,y: x+y)
                            .sortBy(lambda x: x[1], ascending=False)
                       )
rdd_most_active_repo.first()

('ovyx/hammerheadn', 22447)

In [ ]:
# Get the number of Failed 'HTTP request' per 'hour'

In [40]:
# v1 : by year,month,day,hour

failed_req_hr = (rdd.filter(lambda x: "failed" in x.lower())
                 .map(lambda x: (x.split(",")[1].split(":")[0].strip(), 1))
                 .reduceByKey(lambda x,y: x+y)
                 .sortByKey(ascending=True)
                )
failed_req_hr.take(10)

[('2017-03-22T20', 682),
 ('2017-03-23T09', 5232),
 ('2017-03-23T10', 9767),
 ('2017-03-23T11', 10257),
 ('2017-03-23T12', 6791),
 ('2017-03-23T13', 5295),
 ('2017-03-23T14', 5127),
 ('2017-03-23T15', 5090),
 ('2017-03-23T16', 5090),
 ('2017-03-23T17', 5090)]

In [41]:
failed_req_hr.count()

31

In [42]:
failed_req_hr.collect()

[('2017-03-22T20', 682),
 ('2017-03-23T09', 5232),
 ('2017-03-23T10', 9767),
 ('2017-03-23T11', 10257),
 ('2017-03-23T12', 6791),
 ('2017-03-23T13', 5295),
 ('2017-03-23T14', 5127),
 ('2017-03-23T15', 5090),
 ('2017-03-23T16', 5090),
 ('2017-03-23T17', 5090),
 ('2017-03-23T18', 5090),
 ('2017-03-23T19', 5092),
 ('2017-03-23T20', 5090),
 ('2017-03-23T21', 5091),
 ('2017-03-23T22', 5092),
 ('2017-03-23T23', 5088),
 ('2017-03-24T00', 5088),
 ('2017-03-24T01', 5088),
 ('2017-03-24T02', 98),
 ('2017-03-24T03', 98),
 ('2017-03-24T04', 100),
 ('2017-03-24T05', 100),
 ('2017-03-24T06', 98),
 ('2017-03-24T07', 100),
 ('2017-03-24T08', 100),
 ('2017-03-24T09', 97),
 ('2017-03-24T10', 224),
 ('2017-03-24T11', 310),
 ('2017-03-24T12', 709),
 ('2017-03-24T13', 417),
 ('2017-03-24T14', 17)]

In [53]:
# v2: by hour only
from datetime import datetime
def get_failed_hr(line):
    date_t = line.split(",")[1].replace(" ","").replace("T"," ").replace("+00:00"," ").strip()
    dt = datetime.strptime(date_t, "%Y-%m-%d %H:%M:%S")
    return dt.hour

get_failed_hr(str)

10

In [54]:
failed_req_h = (rdd.filter(lambda x: "failed" in x.lower())
                 .map(lambda x: (get_failed_hr(x), 1))
                 .reduceByKey(lambda x,y: x+y)
                 .sortByKey(ascending=True)
                )
failed_req_h.collect()

[(0, 5088),
 (1, 5088),
 (2, 98),
 (3, 98),
 (4, 100),
 (5, 100),
 (6, 98),
 (7, 100),
 (8, 100),
 (9, 5329),
 (10, 9991),
 (11, 10567),
 (12, 7500),
 (13, 5712),
 (14, 5144),
 (15, 5090),
 (16, 5090),
 (17, 5090),
 (18, 5090),
 (19, 5092),
 (20, 5772),
 (21, 5091),
 (22, 5092),
 (23, 5088)]